In [5]:
"""
命令式编程很方便，但效率不高
符号式编程，完全定义了过程之后才执行计算--1.定义计算流程 2.编译为可执行程序 3.给定输入，调用编译好的程序
"""
# 通过命令式编程模拟符号式编程
def add_():
    return """
def add(a, b):
    return a + b
"""

def fancy_func_():
    return """
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
"""

def evoke_():
    return add_() + fancy_func_() + 'print(fancy_func(1, 2, 3, 4))'

prog = evoke_()
print(prog)
y = compile(prog, '', 'exec')
exec(y)


def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
print(fancy_func(1, 2, 3, 4))
10


In [6]:
""" Sequential的混合式编程 """
# 单线程的Python解释器很难让所有GPU都保持忙碌-->将Sequential替换为HybirdSequential来解决这个瓶颈
import torch
from torch import nn
from d2l import torch as d2l

def get_net():
    net = nn.Sequential(
        nn.Linear(512, 256), nn.ReLU(),
        nn.Linear(256, 128), nn.ReLU(),
        nn.Linear(128, 2)
    )
    return net

x = torch.randn(size=(1, 512))
net = get_net()
net(x)

tensor([[0.0203, 0.0116]], grad_fn=<AddmmBackward0>)

In [7]:
# 通过torch.jit.script转换模型，编译和优化计算，计算结果不受影响
net = torch.jit.script(net)
net(x)

tensor([[0.0203, 0.0116]], grad_fn=<AddmmBackward0>)

In [8]:
class Benchmark:
    """ 度量时间的类 """
    def __init__(self, desciption='Done') -> None:
        self.description = desciption
    
    def __enter__(self):
        self.timer = d2l.Timer()
        return self
    
    def __exit__(self, *args):
        print(f'{self.description}:{self.timer.stop():.4f} sec')

In [31]:
net = get_net()
with Benchmark('without torchscript'):
    for _ in range(1000): net(x)

net = torch.jit.script(net)     # 使用符号式编程 提高了计算性能
with Benchmark('with torchscript'):
    for _ in range(1000): net(x)

without torchscript:0.1709 sec
with torchscript:0.1125 sec


In [33]:
""" 序列化 """
""" 编译模型的好处之一--可以将模型及其参数序列化到磁盘，这可以将训练好的模型部署到其他设备上 """
net.save('my_mlp')
!ls -lh my_mlp*

SyntaxError: invalid syntax (1779718487.py, line 4)